In [3]:
import pandas as pd, math, time, json

choices = pd.read_csv(
    "2022_choices.csv",
    index_col="code"
)

votes = pd.read_csv(
    "2022_votes.csv",
    usecols=[1, 2],
    index_col=["voter id"]
)
votes = votes.sample(frac=1)
total_votes = len(votes)
quota = math.ceil(total_votes / 3) + 1

for col in ["first", "ALP", "LP", "Pocock", "exhausted"]:
    choices.loc[:, col] = 0

def sort_prefs(prefs):
    new_prefs = []    
    for x in prefs:
        if choices.at[x, "btl"]:
            new_prefs.append(x)
        else:
            new_prefs.append(choices[(choices["candidateName"] == choices.at[x, "candidateName"]) & (choices["btl"])].index[0])            
    return new_prefs

def three_party_preferred(prefs):
    prefs_3pp = "".join([x for x in prefs if x in "LRT"])
    if len(prefs_3pp) == 0:
        return [0, 0, 0, 1]
    first_prefs_3pp = prefs_3pp[0]
    if first_prefs_3pp == "L":
        return [1, 0, 0, 0]
    elif first_prefs_3pp == "R":
        return [0, 1, 0, 0]
    else:
        return [0, 0, 1, 0]

start = time.time()
interval = 2
print("analysing votes ...", end="\r")
for i, voter in enumerate(votes.index):
    prefs = sort_prefs(votes.at[voter, "prefs"])
    first = prefs[0]
    dist = three_party_preferred(prefs)
    choices.at[first, "first"] = choices.at[first, "first"] + 1
    choices.loc[first, "ALP":"exhausted"] = choices.loc[first, "ALP":"exhausted"] + dist
    if time.time() - interval > start:
        interval = interval + 2
        print(f"analysing votes ... {(i + 1) / total_votes:.1%}", end="\r")
print("analysing votes ... complete")
choices.to_csv("2022_results_actual.csv")

choices_pc = choices.copy()
choices_pc.loc[:, "ALP":"exhausted"] = choices_pc.loc[:, "ALP":"exhausted"] / total_votes
choices_pc.to_csv("2022_results_actual_pc.csv")

choices_pc.loc[:, "ALP":"exhausted"].sum()

analysing votes ... complete


ALP          0.407115
LP           0.271555
Pocock       0.314368
exhausted    0.006963
dtype: float64

In [5]:
choices

,PartyNm,PartyAb,btl,candidateName,first,ALP,LP,Pocock,exhausted
code,,,,,,,,,
A,Australian Labor Party,ALP,False,GALLAGHER Katy,0,0,0,0,0
B,Sustainable Australia Party - Stop Overdevelop...,SPP,False,ANGEL Joy,0,0,0,0,0
C,United Australia Party,UAPP,False,SAVOULIDIS James,0,0,0,0,0
D,Liberal,LP,False,SESELJA Zed,0,0,0,0,0
E,David Pocock,DAVI,False,POCOCK David,0,0,0,0,0
F,Kim for Canberra,KCBR,False,RUBENSTEIN Kim,0,0,0,0,0
G,Australian Progressives,AUP,False,FAULKNER Therese,0,0,0,0,0
H,The Greens,GRN,False,GORENG GORENG Tjanara,0,0,0,0,0
I,Legalise Cannabis Australia,HMP,False,KATELARIS Andrew,0,0,0,0,0
